In [ ]:
import docx
import os
import requests
import streamlit as st
import transformers
import  re


"""Loads the demo script from the given file."""

def load_demo_script(Demo_Script):
    with open(Demo_Script, "r") as f:
        text = f.read()
    return text

def translate_text(text, language):
    
"""Translates the given text to the given language."""

    translator = transformers.BardModel.from_pretrained("bard")
    translated_text = translator(text, target_lang=language)
    return translated_text

def get_names_for_country(country):
    """Gets a list of names from the given country."""
    names_url = f"https://api.namefaker.com/?country={country}&format=json"
    response = requests.get(names_url)
    if response.status_code == 200:
        data = response.json()
        names = [name["name"] for name in data]
    return names
    else:
        print(f"Error getting names for country: {country}")
    return []

def replace_names(text, names):
"""Replaces all occurrences of names in the given text with names from the given list."""
    pattern = r"([A-Z][a-z]+\s[A-Z][a-z]+)"
    for name in names:
    text = re.sub(pattern, name, text)
    return text

def localize_demo_script(filename, language, country):
"""Localizes the demo script in the given file to the given language and country."""
    text = load_demo_script(filename)
    translated_text = translate_text(text, language)
    names = get_names_for_country(country)
    localized_text = replace_names(translated_text, names)
    return localized_text

def save_demo_script(filename, text):
"""Saves the demo script to the given file."""
    doc = docx.Document()
  doc.add_paragraph(text)
  doc.save(filename)

def main():
  """The main function."""
  st.title("Demo Script Localization")

  language = st.selectbox("Language", ["English", "Spanish", "French", "German", "Chinese"])
  country = st.selectbox("Country", ["United States", "United Kingdom", "Canada", "India", "China"])

  filename = st.file_uploader("Upload Demo Script")
  if filename is not None:
    text = localize_demo_script(filename, language, country)
    save_demo_script("localized_demo_script.docx", text)
    st.success("Demo script localized successfully!")

if __name__ == "__main__":
  main()